In [1]:
from tmdbv3api import TMDb
from tmdbv3api import Movie
from tmdbv3api.exceptions import TMDbException
import random
import pandas as pd
import csv
import numpy as np
from math import exp
import ast
tmdb=TMDb()
tmdb.api_key='9cf68f4c97c8f0cc6bb9646da389a808'

In [2]:
# Initialize csv file paths
csv_file_path= 'data_with_mpaa.csv'
# Create DataFrames from both CSV files
df = pd.read_csv(csv_file_path)

#This function merges MPAA data from TMDB and Box Office Mojo, then one hot encodes the merged data. When Data from both sources is avaliable, preference is given to data from Box Office Mojo for the merge. 

In [4]:
#Checks how the ratings are labeled to rename them before the merge
tmdb_ratings = df['MPAA_TMDB'].unique()
bom_ratings = df['MPAA'].unique()
print(tmdb_ratings)
print(bom_ratings)




[nan 'R' 'PG-13' 'PG' 'G' 'NR' 'R ' 'NC-17']
[nan 'R' 'PG-13' 'PG' 'Unrated' 'G' 'NC-17' 'Not Rated']


In [5]:
#Replace 'R ' with 'R' to clean the MPAA ratings from TMDB before the merge
df['MPAA_TMDB'].replace({'R ': 'R'}, inplace=True)
# Replace 'Unrated' or 'Not Rated' with 'NR' in MPAA befoore the merge
df['MPAA'].replace({'Unrated': 'NR', 'Not Rated': 'NR'}, inplace=True)


In [6]:
# Merge the two columns giving preference to Column 2
df['Merged_MPAA'] = df['MPAA'].fillna(df['MPAA_TMDB'])
df['Merged_MPAA'] = df['Merged_MPAA'].fillna('NR')

#Identify unique rating strings
ratings = df['Merged_MPAA'].unique()

#One-hot encode the unique strings
one_hot_encoded = pd.get_dummies(df['Merged_MPAA'], columns=ratings)

#Join the one-hot encoded columns with the original DataFrame
df_output = pd.concat([df, one_hot_encoded], axis=1)

In [7]:
df_output.to_csv('data_with_encoded_mpaa.csv', index = False)